# 🗺️ Mapping

## 📚 Schedule of Notebooks

Complete these notebooks in order. Each shows estimated duration.

1. 🎮 [Simulation Primer](./00-simulation-primer.ipynb) - **60 min**
2. 📍 [Localisation](./01-localisation.ipynb) - **60 min**
3. 🗺️ [Mapping](./02-mapping.ipynb) - **60 min**
4. 🛣️ [SLAM](./03-slam.ipynb) - **80 min**
5. 🛣️ [Path Planning](./04-path-planning.ipynb) - **40 min**

The second problem in navigation is to work out where you want to go. Mapping is the process of building a model of the world around you. This is a fundamental problem in robotics, and is paired with the problem of localization.

## Representing the world

There are many ways to **represent the world**. For 2D environments, the most common is to use a `grid`. This is a 2D array of cells, where each cell represents a small area of the world. The value of each cell is a probability that the cell is occupied. This is called an ***occupancy grid***.

In some situations it is more useful to represent the world as a set of features with relationships between them. This kind of map is called a `graph`. Graphs are useful for representing the topology of the world, and for representing the **relationships between objects** in the world.

### Grid or Graph?

Grids and graphs are both useful representations of the world. Grids are useful for representing the `geometry` of the world, and graphs are useful for representing the `topology` of the world. In practice, you will often use both representations at the same time. Grids are easy to use for `localization`, because you can easily compare the map to the sensor readings. Graphs are easy to use for `planning`, because you can easily find the shortest path between two points. Grids are **memory intensive**, because they require a lot of storage space. Graphs are **computationally intensive**, because they require a lot of computation to find the shortest path.

The best approach is to use both representations at the same time, for example:

- You could use a grid to represent the immediate surroundings of the robot
- a graph to represent the topology of nearby landmarks
- and another lower resolution grid to represent the topology of the whole world.

## The Mapping Procedure

The first step to mapping is to make a measurement of the world. This could be a laser scan, a depth image, or a point cloud. The next step is to mark the measurement on a map. The challenge with this is deciding where to start drawing the measurement on the map. Since the measurement is relative to the robot, you need to know where the robot is on the map. This is the problem of localization. Unfortunately, the problem of localization is also the problem of mapping. This is a chicken and egg problem. You need to know where the robot is to draw the map, but you need to draw the map to know where the robot is.

The solution is to use a probabilistic approach. You can start with a guess of where the robot is, and then update the guess as you make measurements. This is called a Bayes filter.

## Laser Scan Mapping

In a 2 dimensional world, the most common sensor used for mapping is a laser scanner. A laser scanner emits a laser beam and measures the time it takes for the beam to return. This gives you a distance measurement to the nearest object in the direction of the beam. By sweeping the beam around, you can get a 360 degree view of the world around you. This is called a `laser scan`.

### Occupancy Grid Mapping

The most common approach to mapping with a laser scanner is to use an occupancy grid. The occupancy grid is a 2D array of cells, where each cell represents a small area of the world. The value of each cell is a probability that the cell is occupied. The probability is updated using the measurements from the laser scanner.

### Occupancy Grid Mapping Algorithm

The algorithm for occupancy grid mapping is as follows:

1. For each cell in the grid, calculate the probability that the cell is occupied given the measurements from the laser scanner.
2. Update the probability of each cell using Bayes' theorem.
3. move the scanner to a new location by a known amount
4. Repeat steps 1 and 2 with a new measurement from the laser scanner.

> **NOTE:**
>
> **Step 3.** allows us to build a map without having to solve the problem of localization, since we move a well defined amount. This is a common approach to mapping, and is called **odometry-based mapping**.
>
> This approach is not always accurate. Real world approaches uses a **simultaneous localization and mapping (SLAM)**. SLAM will be covered in the next notebook.

### Let's look at an example:

> **Hint:** 
> 
> Run the code cell below

In [ ]:
"""
laser based occupancy grid mapping algorithm example using odometry

In this example, we will simulate a laser scan and use it to update an occupancy grid.
The occupancy grid is a 2D array of cells, where each cell represents a small area of the world.
The value of each cell is a probability that the cell is occupied.
The probability is updated using the measurements from the laser scanner.

The robot moves a certain distance and then takes another laser scan.
"""

import numpy as np
import matplotlib.pyplot as plt

# generate a map of obstacles (ground truth environment)
def generate_obstacle_map(size=20):
    print("Generating obstacle map...")
    # create a size x size grid with obstacles around the perimeter and some internal obstacles
    obstacle_map = np.zeros((size, size))

    # add boundary walls
    obstacle_map[0, :] = 1
    obstacle_map[-1, :] = 1
    obstacle_map[:, 0] = 1
    obstacle_map[:, -1] = 1

    # add some internal obstacles for interest
    obstacle_map[5:8, 10] = 1  # vertical wall
    obstacle_map[12, 5:10] = 1  # horizontal wall
    obstacle_map[15, 15] = 1   # single obstacle

    return obstacle_map

# simulate a laser scan from the robot's position against the obstacle map
def simulate_laser_scan(obstacle_map, robot_x, robot_y, max_range=10):
    # print(f"Simulating laser scan at position ({robot_x}, {robot_y})...")

    # simulate a laser scan with coarse angular resolution for speed
    angles = np.arange(0, 360, 10)  # 36 rays per scan
    distances = []

    for angle in angles:
        dx = np.cos(np.radians(angle))
        dy = np.sin(np.radians(angle))

        # check distance with few steps
        distance = np.nan  # no detection by default
        for step in np.linspace(0, max_range, 10):
            x = int(robot_x + step * dx)
            y = int(robot_y + step * dy)
            if (x < 0 or x >= obstacle_map.shape[0] or
                y < 0 or y >= obstacle_map.shape[1] or
                obstacle_map[x, y] == 1):
                distance = step  # hit an obstacle
                break
        distances.append(distance)

    return angles, np.array(distances)

# update occupancy grid based on laser scan
def update_occupancy_grid(grid, robot_x, robot_y, angles, distances):
    for angle, distance in zip(angles, distances):
        # skip rays that don't hit anything (nan values)
        if np.isnan(distance):
            continue

        # convert polar coordinates to Cartesian coordinates relative to robot
        dx = distance * np.cos(np.radians(angle))
        dy = distance * np.sin(np.radians(angle))

        # calculate absolute position of detected obstacle
        obs_x = int(robot_x + dx)
        obs_y = int(robot_y + dy)

        # update the occupancy grid cell corresponding to the detected obstacle
        if 0 <= obs_x < grid.shape[0] and 0 <= obs_y < grid.shape[1]:
            grid[obs_x, obs_y] += 1  # increment the occupancy probability for that cell

    return grid

# simulate multiple laser scans with robot movement and show iterations
def simulate_mapping_with_iterations():
    print("Starting occupancy grid mapping simulation with iterations...")
    # create the ground truth obstacle map
    obstacle_map = generate_obstacle_map(size=20)

    # create an empty occupancy grid (same size as obstacle map)
    occupancy_grid = np.zeros_like(obstacle_map, dtype=float)

    # robot starts at the center of the map
    robot_positions = [
        (10, 10),  # start position - center
        (10, 12),  # move down
        (10, 14),  # move down more
        (14, 14),  # move right
        (14, 10),  # move up
        (8, 10),   # move left
    ]

    # create subplots to show the map building up over iterations
    fig, axes = plt.subplots(2, 3, figsize=(14, 8))
    axes = axes.flatten()

    for i, (robot_x, robot_y) in enumerate(robot_positions):
        # simulate a laser scan from the current robot position
        angles, distances = simulate_laser_scan(obstacle_map, robot_x, robot_y)

        # update the occupancy grid
        occupancy_grid = update_occupancy_grid(occupancy_grid, robot_x, robot_y, angles, distances)

        # plot the current state of the occupancy grid
        ax = axes[i]
        ax.imshow(occupancy_grid, cmap='gray_r', interpolation='nearest', origin='lower', vmin=0, vmax=3)

        # plot laser scan points in red
        for angle, distance in zip(angles, distances):
            if not np.isnan(distance):
                dx = distance * np.cos(np.radians(angle))
                dy = distance * np.sin(np.radians(angle))
                scan_x = robot_x + dx
                scan_y = robot_y + dy
                ax.plot(scan_y, scan_x, 'r.', markersize=4)

        ax.plot(robot_y, robot_x, 'ro', markersize=8)
        ax.set_title(f'Iteration {i+1}', fontsize=10)
        ax.set_xticks([])
        ax.set_yticks([])

    plt.suptitle('Occupancy Grid Mapping Over Time', fontsize=12)
    plt.tight_layout()
    plt.show()

    print("✓ Complete!")

    # also show the ground truth for comparison
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

    ax1.imshow(obstacle_map, cmap='binary', interpolation='nearest', origin='lower')
    ax1.set_title('Ground Truth Obstacles', fontsize=11)
    ax1.set_xticks([])
    ax1.set_yticks([])

    ax2.imshow(occupancy_grid, cmap='gray_r', interpolation='nearest', origin='lower', vmin=0, vmax=3)

    # plot all laser scan points on final map
    for robot_x, robot_y in robot_positions:
        angles, distances = simulate_laser_scan(obstacle_map, robot_x, robot_y)
        for angle, distance in zip(angles, distances):
            if not np.isnan(distance):
                dx = distance * np.cos(np.radians(angle))
                dy = distance * np.sin(np.radians(angle))
                scan_x = robot_x + dx
                scan_y = robot_y + dy
                ax2.plot(scan_y, scan_x, 'r.', markersize=3)

    ax2.set_title('Final Scanned Map', fontsize=11)
    ax2.set_xticks([])
    ax2.set_yticks([])

    plt.tight_layout()
    plt.show()

    print("✓ Simulation finished!")

# run the simulation
simulate_mapping_with_iterations()


### What This Example Demonstrates

The example above shows:

1. **Ground Truth Environment**: A predefined obstacle map with walls and internal obstacles that the robot will scan.

2. **Realistic Laser Scanning**: The laser scanner casts rays from the robot's position and detects obstacles in the environment.

3. **Robot Movement**: The robot moves through 6 different positions, taking a laser scan at each location.

4. **Incremental Map Building**: The 6 subplots show how the occupancy grid builds up as the robot explores:
   - Initially, only obstacles near the starting position are detected
   - As the robot moves, it discovers more of the environment
   - Multiple detections of the same obstacle increase its probability (darker = more detections)

5. **Odometry-Based Mapping**: The robot knows exactly where it moves (perfect odometry), so it can correctly place each laser scan in the map without localization uncertainty.

6. **Comparison**: The final plots compare the ground truth (actual obstacles) with the accumulated occupancy grid (what the robot has discovered).

Notice how the occupancy grid doesn't perfectly match the ground truth because:
- The robot hasn't explored all areas
- Laser scans have limited range and uncertainty
- Some obstacles may be occluded by others

---

## Review Questions

Test your understanding of occupancy grid mapping:

1. **What is the main advantage of using an occupancy grid over a feature-based (graph) representation?**
   - How would you represent a completely open, featureless hallway with each approach?

2. **In the example above, the robot has perfect odometry (knows exactly where it moved). What happens in real robots that don't have perfect odometry?**
   - How would errors in odometry affect the final map?

3. **Why are some obstacles not detected even though the robot passed nearby them?**
   - Hint: Look at the laser scan range and check which areas remain white in the final map.

4. **The occupancy grid is updated by incrementing cells where obstacles are detected. How would you modify this to include uncertainty?**
   - What would happen if you used probabilistic updates (log-odds) instead of simple counting?

5. **In real-world robotics, mapping is coupled with localization (SLAM - Simultaneous Localization and Mapping).**
   - Why can't we just use pure odometry for long-term mapping missions?
   - What problems arise from odometry drift?

6. **Challenge Question: How would you detect and handle dynamic obstacles (moving objects) in occupancy grid mapping?**
   - What timestamps or temporal information would be useful?